In [1]:
library(coin)
library(lme4)
library(lmerTest)
library(plyr)
library(xtable)
library(LMERConvenienceFunctions)
library(car)

Loading required package: survival
Loading required package: Matrix

Attaching package: 'lmerTest'

The following object is masked from 'package:lme4':

    lmer

The following object is masked from 'package:stats':

    step

Loading required package: carData
Registered S3 methods overwritten by 'car':
  method                          from
  influence.merMod                lme4
  cooks.distance.influence.merMod lme4
  dfbeta.influence.merMod         lme4
  dfbetas.influence.merMod        lme4


In [2]:
get_processed_data <- function(data_path){
    data <- read.table(file.path(data_path, "measures.csv"), header = TRUE, sep = ",")
    data$subj_id <- as.factor(data$subj_id)
    
    data$tta_condition <- scale(data$tta_condition)
    data$d_condition <- scale(data$d_condition)
    data$v_condition <- scale(data$v_condition)
    data$RT <- scale(data$RT)
    
    data$decision <- relevel(data$decision, ref=2)
  
    return(data)
}

data <- get_processed_data("../data")

In [3]:
contrasts(data$decision) 

,Go
Stay,0
Go,1


In [4]:
data

subj_id,session,route,intersection_no,idx_bot_spawn,idx_response,idx_min_distance,min_distance,RT,is_go_decision,is_collision,decision,tta_condition,d_condition,v_condition
<fct>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,"<dbl[,1]>",<fct>,<fct>,<fct>,"<dbl[,1]>","<dbl[,1]>","<dbl[,1]>"
129,1,1,5,120,161,471,3.226016,-0.7196088,True,True,Go,-1.217441821,-1.222303745,-0.33196290
129,1,1,6,46,56,389,16.099580,-0.9231816,True,False,Go,0.005859346,-1.222303745,-1.01416299
129,1,1,8,50,87,506,3.234011,-0.7450874,False,False,Stay,0.005859346,-1.222303745,-1.01416299
129,1,1,10,84,124,430,8.046337,-0.7263946,True,False,Go,-1.217441821,1.226213032,1.94203742
129,1,1,11,91,92,380,48.915927,-0.9829731,True,False,Go,1.229160512,0.001954644,-0.71096295
129,1,1,13,56,111,414,33.930989,-0.6275529,True,False,Go,1.229160512,0.001954644,-0.71096295
129,1,1,14,3,68,393,3.625540,-0.5241021,False,False,Stay,-1.217441821,1.226213032,1.94203742
129,1,1,15,62,102,403,7.893319,-0.7262025,True,False,Go,-1.217441821,0.001954644,0.80503726
129,1,1,19,65,125,459,12.732931,-0.5946484,True,False,Go,0.005859346,0.001954644,-0.10456287


# Decision as a function of TTA and d

In [5]:
rnd_effects_analysis_decision <- function(data){
    rnd1 = glmer(decision ~ tta_condition + d_condition + (1|subj_id), data, family = binomial)
    rnd2 = glmer(decision ~ tta_condition + d_condition + (d_condition|subj_id), data, family = binomial)
    rnd3 = glmer(decision ~ tta_condition + d_condition + (tta_condition|subj_id), data, family = binomial)
    rnd4 = glmer(decision ~ tta_condition + d_condition + (d_condition + tta_condition|subj_id), data, family = binomial)
    
    rnd.anova = anova(rnd1, rnd2, rnd3, rnd4)
    print(rnd.anova)
    
    print("Best model according to AIC")
    print(row.names(rnd.anova[rnd.anova$AIC==min(rnd.anova$AIC), ]))
    print("Best model according to BIC")
    print(row.names(rnd.anova[rnd.anova$BIC==min(rnd.anova$BIC), ]))
}

In [6]:
rnd_effects_analysis_decision(data)

Data: data
Models:
rnd1: decision ~ tta_condition + d_condition + (1 | subj_id)
rnd2: decision ~ tta_condition + d_condition + (d_condition | subj_id)
rnd3: decision ~ tta_condition + d_condition + (tta_condition | subj_id)
rnd4: decision ~ tta_condition + d_condition + (d_condition + tta_condition | 
rnd4:     subj_id)
     Df    AIC    BIC  logLik deviance  Chisq Chi Df Pr(>Chisq)    
rnd1  4 1782.9 1805.0 -887.45   1774.9                             
rnd2  6 1743.5 1776.7 -865.74   1731.5 43.418      2  3.732e-10 ***
rnd3  6 1757.5 1790.8 -872.76   1745.5  0.000      0          1    
rnd4  9 1733.8 1783.6 -857.88   1715.8 29.772      3  1.541e-06 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
[1] "Best model according to AIC"
[1] "rnd4"
[1] "Best model according to BIC"
[1] "rnd2"


In [7]:
fitted_model = glmer(decision ~ tta_condition + d_condition + (d_condition + tta_condition | subj_id), data, family = binomial)
print(summary(fitted_model))

output = summary(fitted_model)$coefficients

file_name = file.path("output", "tab_decision.tex")
print(xtable(output, display = c("g","g","g","g","g"), # digits = c(2,3,2,2,2,-2), 
             label = "tab:decision", 
             caption = "Results of statistical analysis of the effect of distance and TTA conditions on decision. The mixed-effects model included random slopes of distance and TTA conditions per participant."), 
      caption.placement = "top", table.placement="!h", math.style.exponents = TRUE, floating.environment = "table*", 
      type = "latex", file = file_name, booktabs = TRUE)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
decision ~ tta_condition + d_condition + (d_condition + tta_condition |  
    subj_id)
   Data: data

     AIC      BIC   logLik deviance df.resid 
  1733.8   1783.6   -857.9   1715.8     1870 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.3153 -0.4792 -0.1138  0.5267  5.8247 

Random effects:
 Groups  Name          Variance Std.Dev. Corr       
 subj_id (Intercept)   1.1606   1.0773              
         d_condition   0.2572   0.5072   -0.42      
         tta_condition 0.1277   0.3574    0.21 -0.58
Number of obs: 1879, groups:  subj_id, 16

Fixed effects:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept)    -0.2827     0.2789  -1.014    0.311    
tta_condition   1.0445     0.1141   9.153   <2e-16 ***
d_condition     1.3984     0.1492   9.371   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' '

In [8]:
coef(fitted_model)

$subj_id
    (Intercept) tta_condition d_condition
129 -0.12284651     1.1649986   1.0033181
280 -1.42374381     1.1031520   1.7237542
389 -0.95841722     1.0617398   1.7140360
421 -0.02528099     0.5786331   1.2931959
525  0.39222477     1.1284551   0.9337994
616 -1.56161127     1.0304478   1.5923785
618 -0.44535240     1.1917372   1.3126806
642 -0.97815654     0.5902438   1.7285279
652 -1.36081620     1.5233962   0.7988719
755  0.08239682     0.7075335   1.6927423
776  1.02780788     1.1153138   1.3381346
827  1.26763931     1.3436868   0.9022874
853 -2.29360257     0.6630716   2.0246826
913  0.65513424     1.5143724   0.7237592
969  0.37751788     0.7324759   2.2405456
996  0.97664917     1.1498615   1.0318904

attr(,"class")
[1] "coef.mer"

# RT as a function of TTA and distance

In [9]:
rnd_effects_analysis_RT <- function(data){
    rnd1 = lmer(RT ~ tta_condition + d_condition + (1|subj_id), data)
    rnd2 = lmer(RT ~ tta_condition + d_condition + (d_condition|subj_id), data)
    rnd3 = lmer(RT ~ tta_condition + d_condition + (tta_condition|subj_id), data)
    rnd4 = lmer(RT ~ tta_condition + d_condition + (d_condition + tta_condition|subj_id), data)
    
    rnd.anova = anova(rnd1, rnd2, rnd3, rnd4)
    print(rnd.anova)
    
    print("Best model according to AIC")
    print(row.names(rnd.anova[rnd.anova$AIC==min(rnd.anova$AIC), ]))
    print("Best model according to BIC")
    print(row.names(rnd.anova[rnd.anova$BIC==min(rnd.anova$BIC), ]))
}

In [10]:
rnd_effects_analysis_RT(data[data$decision=="Go",])

boundary (singular) fit: see ?isSingular
boundary (singular) fit: see ?isSingular
refitting model(s) with ML (instead of REML)


Data: data
Models:
rnd1: RT ~ tta_condition + d_condition + (1 | subj_id)
rnd2: RT ~ tta_condition + d_condition + (d_condition | subj_id)
rnd3: RT ~ tta_condition + d_condition + (tta_condition | subj_id)
rnd4: RT ~ tta_condition + d_condition + (d_condition + tta_condition | 
rnd4:     subj_id)
     Df     AIC     BIC logLik deviance  Chisq Chi Df Pr(>Chisq)  
rnd1  5 -951.16 -927.24 480.58  -961.16                           
rnd2  7 -951.85 -918.37 482.93  -965.85 4.6984      2    0.09544 .
rnd3  7 -949.26 -915.77 481.63  -963.26 0.0000      0    1.00000  
rnd4 10 -947.30 -899.47 483.65  -967.30 4.0482      3    0.25630  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
[1] "Best model according to AIC"
[1] "rnd2"
[1] "Best model according to BIC"
[1] "rnd1"


Model with rnd2 fails to converge, so using random intercept only

In [11]:
fitted_model = lmer(RT ~ tta_condition + d_condition + (1 | subj_id), data[data$decision=="Go",])
print(summary(fitted_model))

output = summary(fitted_model)$coefficients

file_name = file.path("output", "tab_RT.tex")
print(xtable(output, display = c("g","g","g","g","g","g"), # digits = c(2,3,2,2,2,-2), 
             label = "tab:RT", 
             caption = "Results of statistical analysis of the effect of distance and TTA conditions on response time in ``go'' decisions. The mixed-effects model included random intercept per participant."), 
      caption.placement = "top", table.placement="!h", math.style.exponents = TRUE, floating.environment = "table*", 
      type = "latex", file = file_name, booktabs = TRUE)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: RT ~ tta_condition + d_condition + (1 | subj_id)
   Data: data[data$decision == "Go", ]

REML criterion at convergence: -937.8

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.7676 -0.5920 -0.1078  0.4224  8.1963 

Random effects:
 Groups   Name        Variance Std.Dev.
 subj_id  (Intercept) 0.006438 0.08024 
 Residual             0.018757 0.13696 
Number of obs: 883, groups:  subj_id, 16

Fixed effects:
                Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)    -0.628820   0.020962  15.686854 -29.998 2.80e-15 ***
tta_condition   0.020211   0.005071 869.329015   3.986 7.29e-05 ***
d_condition    -0.008337   0.005414 873.577436  -1.540    0.124    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Correlation of Fixed Effects:
            (Intr) tt_cnd
tta_conditn -0.111       
d_condition -0.148  0.165


In [12]:
coef(fitted_model)

$subj_id
    (Intercept) tta_condition  d_condition
129  -0.4918033    0.02021083 -0.008337139
280  -0.5509421    0.02021083 -0.008337139
389  -0.6252943    0.02021083 -0.008337139
421  -0.6696222    0.02021083 -0.008337139
525  -0.7531898    0.02021083 -0.008337139
616  -0.6092438    0.02021083 -0.008337139
618  -0.6158034    0.02021083 -0.008337139
642  -0.5987878    0.02021083 -0.008337139
652  -0.4533160    0.02021083 -0.008337139
755  -0.6669476    0.02021083 -0.008337139
776  -0.6799241    0.02021083 -0.008337139
827  -0.6732346    0.02021083 -0.008337139
853  -0.6101907    0.02021083 -0.008337139
913  -0.6839608    0.02021083 -0.008337139
969  -0.6951919    0.02021083 -0.008337139
996  -0.6836662    0.02021083 -0.008337139

attr(,"class")
[1] "coef.mer"